# Модуль 1. Вступ до LLM та RAG

## 1. Пошук та отримання інформації

### Імпорт пошукової системи



Для пошуку у файлах FAQ курсів Zoomcamp ми спочатку будемо використовувати просту пошукову систему, що працює в пам'яті. Відповідна бібліотека знаходиться у файлі minsearch.py. А файл, у свою чергу, знаходиться в репозиторії на GitHub. Ми завантажуємо його як пакет та підключаємо.

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py

--2025-06-12 11:34:43--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/refs/heads/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4073 (4.0K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.98K  --.-KB/s    in 0s      

2025-06-12 11:34:43 (37.8 MB/s) - ‘minsearch.py’ saved [4073/4073]



In [3]:
import minsearch

### Завантаження та обробка даних з файлів FAQ

Усі три документи FAQ вже підготовлені та знаходяться у файлі documents.json. Завантажуємо його з репозиторію. Але спочатку підключимо бібліотеку JSON для роботи з цим форматом файлів.

In [1]:
import json

In [2]:
!wget https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json

--2025-06-13 06:49:17--  https://raw.githubusercontent.com/DataTalksClub/llm-zoomcamp/refs/heads/main/01-intro/documents.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.109.133, 185.199.110.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.109.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 658332 (643K) [text/plain]
Saving to: ‘documents.json.1’

documents.json.1    100%[===================>] 642.90K  --.-KB/s    in 0.008s  

2025-06-13 06:49:18 (75.4 MB/s) - ‘documents.json.1’ saved [658332/658332]



Відкриваємо та читаємо файл documents.json.

In [2]:
with open('documents.json', 'rt') as f_in:
  docs_row = json.load(f_in)

Цей файл має складну вкладену структуру. Спростимо її. Перетворимо її на список словників. Назвемо цей перелік documents. У кожному словнику міститься питання, відповідь на нього та назва розділу FAQa, до якого він відноситься. Додамо також у кожен словник поле course, що містить назву FAQa, до якого він відноситься.

In [3]:
documents = []

for course_dict in docs_row:
  for doc in course_dict['documents']:
    doc['course'] = course_dict['course']
    documents.append(doc)

In [10]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

### Індексація та пошук

У пошуковій бібліотеці є клас Index. Для створення індексу потрібно визначити:

- Text fields — поля, у яких буде здійснюватися пошук (наприклад, question, text, section)
- Keyword fields — поля, за якими можна фільтрувати (наприклад, course)

За ключовими полями можна здійснювати фільтрацію результатів так само, як здійснюється SQL-запит. Наприклад: SELECT * WHERE course = 'Data Engineering Zoomcamp'. За текстовими полями ми будемо здійснювати пошук.

In [9]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [10]:
index.fit(documents)

Нашу базу проіндексовано.

### Налаштування параметрів пошуку та отримання результатів

Сформулюємо якесь питання щодо курсів та збережемо його в змінній.

In [ ]:
q = 'The course has already starded, can I still enroll?'

Налаштуємо параметри пошуку. Спочатку встановимо приорітети.

In [ ]:
boost = {'question': 3.0, 'section': 0.5}

Ми задаємо коефіцієнти ваги для різних полів. Значення за замовчуванням - 1. Полю 'question' ми надаємо коефіцієнт рівний 3, а полю 'section' - 0.5. Це означає, що поле, де зберігається питання для нас в три рази важливіше ніж поле з текстом відповіді. Ми робимо пошук саме по питанню. А от поле з назвою секції в половину менш важливе.

Це допомагає зробити пошук більш релевантним, надаючи більшого значення пошуковим термінам, знайденим у певних полях (наприклад, у заголовках чи запитаннях).

Отримаємо результат пошуку.

In [ ]:
results = index.search(
    query=q,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
)

У параметрі filter_dict ми встановлюємо фільтр по полю course. Тобто шукати відповідь питання ми будемо тільки в курсі data-engineering-zoomcamp. Останній параметр num_results встановлює кількість результатів у виводі.

In [ ]:
results

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

## 2. Генерація відповіді за допомогою GPT-4o

Ми проіндексували документи за допомогою пошукової системи. Тепер ми можемо використати отриману нами з бази даних інформацію і помістити її в запит користувача у якості контексту.

Спочатку імпотруємо бібліотеку OpenAI.

In [32]:
 from openai import OpenAI

Тепер створимо клієнт.

In [33]:
client = OpenAI()

OpenAIError: The api_key client option must be set either by passing api_key to the client or by setting the OPENAI_API_KEY environment variable

Нам не треба вказувати АРІ-ключ якщо ми зберегли його в змінній оточення.

Тепер створимо шаблон промпта.

In [ ]:
prompt_template = """
You are a course teaching assistant.
Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output "No information found".

QUESTION: {question}
CONTEXT:
{context}
"""

У шаблоні ми використовуємо змінні {question} та {context}. На їх місце буде підставлено потрібний контент.

Сформуємо контекст.

In [ ]:
context = ""
for doc in results:
    context += f"Section: {doc['section']}\n"
    context += f"Question: {doc['question']}\n"
    context += f"Answer: {doc['text']}\n\n"

In [ ]:
print(context)

Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.

Section: General course-related questions
Question: Course - When will the course start?
Answer: The purpose of this document is to capture frequently asked technical questions
The exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start wit

Створюємо промпт на основі шаблону.

In [ ]:
prompt = prompt_template.format(question=q, context=context).strip()

Тут метод strip() використовується для видалення можливих символів пробілу на початку і в кінці рядку. Ці зайві символи можуть погано вплинути на відповідь LLM.

In [ ]:
print(prompt)

You are a course teaching assistant. 
Answer the QUESTION based on the CONTEXT from the FAQ database. 
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doesn't contain the answer, output "No information found".

QUESTION: The course has already starded, can I still enroll?
CONTEXT: 
Section: General course-related questions
Question: Course - Can I still join the course after the start date?
Answer: Yes, even if you don't register, you're still eligible to submit the homeworks.
Be aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.

Section: General course-related questions
Question: Course - Can I follow the course after it finishes?
Answer: Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.
You can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also sta

Викликаємо LLM, передаємо їй сформований нами промпт та отримуємо відповідь.

In [ ]:
response = client.chat.completions.create(
  model="gpt-4o",
  messages=[{"role": "user", "content": prompt}]
)

response.choices[0].message.content

## 3. Поліпшення коду. Створення модулів

У цій частині модуля ми впорядкуємо код, який створили вище. Ми вже проіндексували все, отримали результати, побудували промпт і надіслали його до OpenAI. Тепер треба зібрати все докупи. Ми створимо модульну структуру.

Спочатку створиму функцію search, яка приймає query. Перенесемо в неї частину коду з розділу "Налаштування параметрів пошуку та отримання результатів". Тільки замість q буде query.

In [11]:
def search(query):
  boost = {'question': 3.0, 'section': 0.5}

  results = index.search(
    query=query,
    filter_dict={'course': 'data-engineering-zoomcamp'},
    boost_dict=boost,
    num_results=5
  )

  return results

Створимо функцію build_prompt яка приймає запит query та context (або search_results) і повертає промпт.

In [26]:
def build_prompt(query, search_results):
  prompt_template = """
  You are a course teaching assistant.
  Answer the QUESTION based on the CONTEXT from the FAQ database.
  Use only the facts from the CONTEXT when answering the QUESTION.
  If the CONTEXT doesn't contain the answer, output "No information found".

  QUESTION: {question}
  CONTEXT:
  {context}
  """.strip()

  context = ""

  for doc in search_results:
    context += f"Section: {doc['section']}\n"
    context += f"Question: {doc['question']}\n"
    context += f"Answer: {doc['text']}\n\n"

  prompt = prompt_template.format(question=query, context=context).strip()
  return prompt

Створимо функцію llm яка буде приймати промпт і повертати відповідь LLM.

In [27]:
def llm(prompt):
  response = client.chat.completions.create(
  model="gpt-4o",
  messages=[{"role": "user", "content": prompt}]
  )

  return response.choices[0].message.content

В нас є три функції, що виконують три окремі кроки конвеєра RAG. Одна виконує пошук по базі. Наступна створює промпт. І остання приймає промпт і повертає відповідь від LLM. Залишилося зібрати їх разом в одну функцію, щоб не викликати окремо.

In [ ]:
query = "Haw do I run Kafka?"

def rag(query):
  search_results = search(query)
  prompt = build_prompt(query, search_results)
  answer = llm(prompt)

  return answer

## 4. Реалізація пошуку з Elasticsearch

У цьому розділі ми замінимо простий пошуковий рушій на Elasticsearch. 

### Запуск Elasticsearch в Docker

Ми зараз перебуваємо в середовищі CodeSpaces. Запускаємо Elasticsearch в Doker (який вже встановлено в середовищі). Для цього в терміналі вводимо такий код (попередньо відкриваємо новий термінал, бо в першому вже запущений Jupyter Notebook):

docker run -it \
    --rm \
    --name elasticsearch \
    -m 4GB \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    docker.elastic.co/elasticsearch/elasticsearch:8.4.3

Якщо попередня команда не спрацює, треба спробувати запустити Elasticsearch безпосередньо з Docker Hub. Ось так:

docker run -it \
    --rm \
    --name elasticsearch \
    -p 9200:9200 \
    -p 9300:9300 \
    -e "discovery.type=single-node" \
    -e "xpack.security.enabled=false" \
    elasticsearch:8.4.3

Ми завантажили образ Docker і запустили Elasticsearch. Він працює на портах 9200 та 9300. Щоб перевірити ми відкриваємо ще один новий термінал і вводимо простий запит:

curl http://localhost:9200

Отримуємо відповідь. Це означає, що все працює.

Насправді нам не потрібно, щоб цей порт перенаправлявся на нашу локальну машину, тому що ми будемо використовувати Elasticsearch лише в CodeSpaces, але в будь-якому випадку, він його перенаправляє.

### Індексація документів за допомогою Elasticsearch

In [ ]:
Особливість Elasticsearch полягає в тому, що він є постійним. Коли ми завершуємо процес Jupyter Notebook, minsearch не зберігає 
дані і нам потрібно наступного разу заново будувати індекс. Elasticsearch зберігає всі дані на диску. Наступного разу, коли ми запустимо 
його, у нас будуть всі необхідні дані і нам не потрібно буде його виконувати.
    
Звісно, це залежить від того, як саме ми його запускаємо. Нам може знадобитися виконати монтування томів для цього. Це виходить 
за рамки цього курсу. Більше інформації про Docker можна знайти в курсі з інженерії даних, модуль 1.

Отже, проіндексуємо документи за допомогою Elasticsearch щоб згодом використовувати їх для пошуку. 

Але спочатку розберемось, як використовувати Elasticsearch в Jupyter Notebook. Щоб працювати з Elasticsearch, нам треба його 
підключити. Зробимо це. 

In [5]:
from elasticsearch import Elasticsearch

Створимо клиєнта Elasticsearch.

In [6]:
es_client = Elasticsearch('http://localhost:9200')

Якщо розгортаєш Elasticsearch десь у хмарі, то треба просто використовувати URL-адресу, яку отримаєш. Але в даному випадку в нас локальне розгортання.

Перевіримо, чи працює.

In [7]:
es_client.info()

ObjectApiResponse({'name': 'a9a29fd3c541', 'cluster_name': 'docker-cluster', 'cluster_uuid': 'QmWE3tlDRb2fopjY3OadQA', 'version': {'number': '8.4.3', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '42f05b9372a9a4a470db3b52817899b99a76ee73', 'build_date': '2022-10-04T07:17:24.662462378Z', 'build_snapshot': False, 'lucene_version': '9.3.0', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'})

Створимо індекс Elasticsearch. Індекс - це як таблиця в реляційній базі даних. 

In [8]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

Це налаштування індексу. Тут цікава річ — це властивості (properties). Сourse є ключовим словом. По ньому ми будемо виконувати фільтрацію. Решта типів — text. Це дуже схоже на те, що ми робили з Minsearch, де перші три були text, а course був ключовим словом. 

Тепер нам потрібно дати назву індексу. Назвемо його "course_questions". І нам потрібно буде використовувати клієнт Elasticsearch для створення індексу. Ось так:

In [9]:
index_name = "course-questions"

es_client.indices.create(index=index_name, body=index_settings)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

Індекс створено. Тепер ми можемо індексувати дані за його допомогою. Це займе деякий час. Тому є сенс зробити прогрес-бар. Для цього ми імпортуємо бібліотеку tqdm.

In [11]:
from tqdm.auto import tqdm

В нас вже є список documents, в якому містяться FAQи. Використовуємо цикл для їх перебору і щоб працював прогрес-бар пишемо так: tqdm(documents).

In [12]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

  0%|          | 0/948 [00:00<?, ?it/s]

## Виконання запитів до Elasticsearch

In [13]:
query = "The course has already starded, can I still enroll?"

Запит має доволі складний вигляд. Він був підготовлений заздалегіть. 

In [14]:
search_query = {
    "size": 5,
    "query": {
        "bool": {
            "must": {
                "multi_match": {
                    "query": query,
                    "fields": ["question^3", "text", "section"],
                    "type": "best_fields"
                }
            },
            "filter": {
                "term": {
                    "course": "data-engineering-zoomcamp"
                }
            }
        }
    }
}

Ми вказуємо поля, по яких повинен відбуватися пошук: "question^3", "text" та "section". Звернімо увагу на ^3. Цей запис вказує, що поле question у три рази важливіше ніж інші поля. 

Параметр size вказує, скільки результатів пошуку треба вивести.

Параметр course, це фільтр, який обмежує пошук тільки документом "Data Engineering Zoomcamp". 

Виконаємо запит.

In [15]:
es_client.search(index=index_name, body=search_query) 

ObjectApiResponse({'took': 33, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 401, 'relation': 'eq'}, 'max_score': 48.763668, 'hits': [{'_index': 'course-questions', '_id': 'KEzOaJcBJ9aJY20iM1pe', '_score': 48.763668, '_source': {'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.", 'section': 'General course-related questions', 'question': 'Course - Can I still join the course after the start date?', 'course': 'data-engineering-zoomcamp'}}, {'_index': 'course-questions', '_id': 'KkzOaJcBJ9aJY20iM1pt', '_score': 36.514423, '_source': {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and sylla

Отримали доволі складний об'єкт. Запишемо цей результат у змінну.

In [16]:
response = es_client.search(index=index_name, body=search_query)

Нам треба витягнути з цього об'єкта результати пошуку.

In [18]:
result_docs = []

for hit in response['hits']['hits']:
    result_docs.append(hit['_source'])

In [19]:
result_docs

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at

Тепер зберемо весь цей код у функцію.

In [22]:
def elastic_search(query):
    search_query = {
        "size": 5,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^3", "text", "section"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "data-engineering-zoomcamp"
                    }
                }
            }
        }
    }    

    response = es_client.search(index=index_name, body=search_query)

    result_docs = []

    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])


    return result_docs

In [30]:
def rag(query):
  search_results = elastic_search(query)
  prompt = build_prompt(query, search_results)
  answer = llm(prompt)

  return answer

In [31]:
rag(query)

NameError: name 'client' is not defined